In [ ]:
from pathlib import Path
import shutil
from typing import Iterable, Tuple
import shutil, os

In [49]:

## Renames folders (deepest first) and selected file types inside ROOT, replacing every occurrence of needle with repl.
# Safeguard: for files, it skips the rename if the target already exists to avoid clobbering.

ROOT = Path(r"F:\Alejandro\2p_data\Exp_1_flickering")
needle, repl = "f01", "L453_f11"
exts = {".tif", ".tiff", ".csv", ".py",".json"}

for d in sorted((p for p in ROOT.rglob("*") if p.is_dir()), key=lambda p: len(p.parts), reverse=True):
    if needle in d.name:
        d.rename(d.with_name(d.name.replace(needle, repl)))

for f in (p for p in ROOT.rglob("*") if p.is_file() and p.suffix.lower() in exts and needle in p.name):
    target = f.with_name(f.name.replace(needle, repl))
    if not target.exists():
        f.rename(target)

In [ ]:
# Prints the relative paths under root up to max_depth.
#Use this to sanity-check that renames/moves produced the expected structure.
root = Path(r"C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L453_f10_Exp_1_flickering")
max_depth = 3
# change this to show deeper levels

for p in root.rglob("*"):
    depth = len(p.relative_to(root).parts)
    if depth <= max_depth:
        print(p.relative_to(root))


In [ ]:
# Reorganize legacy experiments into a new standardized tree
#  Given SRC_ROOT (legacy layout) and DST_ROOT (new layout), this:
#  Creates the destination folder structure (TREE).
#  Moves content according to MAP (source subfolder → destination subfolder).
#  Special-cases anatomy: moves items containing “anatomy” into 01_raw/2p/anatomy first.
#  Handles filename collisions by appending _1, _2, … so nothing is overwritten.
#  Processes all experiments in SRC_ROOT whose names start with L.Move files from one directory structure to another

SRC_ROOT = Path(r"F:\Alejandro\2p_data\Exp_1_flickering")
DST_ROOT = Path(r"F:\Data\Alejandro")

TREE = [
    "01_raw",
    "01_raw/2p/anatomy",
    "01_raw/2p/functional",
    "01_raw/2p/metadata",
    "01_raw/confocal/round1",
    "01_raw/confocal/roundn",
    "02_reg/00_preprocessing",
    "02_reg/00_preprocessing/2p_anatomy",
    "02_reg/00_preprocessing/2p_functional/01_individualPlanes",
    "02_reg/00_preprocessing/2p_functional/02_motionCorrected",
    "02_reg/00_preprocessing/r1",
    "02_reg/00_preprocessing/rn",
    "02_reg/01_r1-2p/logs",
    "02_reg/01_r1-2p/matrices",
    "02_reg/02_rn-r1/logs",
    "02_reg/02_rn-r1/transMatrices",
    "02_reg/03_rn-2p/logs",
    "02_reg/03_rn-2p/transMatrices",
    "02_reg/04_r1-ref/logs",
    "02_reg/04_r1-ref/transMatrices",
    "02_reg/05_r2-ref/logs",
    "02_reg/05_r2-ref/transMatrices",
    "02_reg/06_total-ref/logs",
    "02_reg/06_total-ref/transMatrices",
    "02_reg/07_2pf-a/logs",
    "02_reg/07_2pf-a/transMatrices",
    "02_reg/08_2pa-ref/logs",
    "02_reg/08_2pa-ref/transMatrices",
    "03_analysis/functional/suite2P",
    "03_analysis/structural/cellpose",
    "04_plots",
]

# fixed mapping: source -> destination (relative to experiment folder)
MAP = {
    "00_raw": "01_raw/2p/functional",
    "01_metadata": "01_raw/2p/metadata",
    "02_preprocessed": "02_reg/00_preprocessing/2p_functional/01_individualPlanes",
    "03_motion_corrected": "02_reg/00_preprocessing/2p_functional/02_motionCorrected",
    "04_segmented": "03_analysis/functional/suite2P",
}

def make_tree(root: Path):
    for d in TREE:
        (root / d).mkdir(parents=True, exist_ok=True)

def move_contents(src_dir: Path, dst_dir: Path):
    dst_dir.mkdir(parents=True, exist_ok=True)
    for item in src_dir.iterdir():
        if item.name == ".DS_Store":
            continue
        try:
            if item.is_dir():
                shutil.move(str(item), str(dst_dir / item.name))
            else:
                dest = dst_dir / item.name
                if dest.exists():
                    stem, suf = dest.stem, dest.suffix
                    k = 1
                    while (dst_dir / f"{stem}_{k}{suf}").exists():
                        k += 1
                    dest = dst_dir / f"{stem}_{k}{suf}"
                shutil.move(str(item), str(dest))
        except Exception as e:
            print(f"! {item} -> {dst_dir} ({e})")

def move_anatomy(src_raw: Path, dst_exp: Path):
    dst_anatomy = dst_exp / "01_raw/2p/anatomy"
    dst_anatomy.mkdir(parents=True, exist_ok=True)
    for item in src_raw.iterdir():
        if "anatomy" in item.name.lower():
            try:
                if item.is_dir():
                    shutil.move(str(item), str(dst_anatomy / item.name))
                else:
                    shutil.move(str(item), str(dst_anatomy / item.name))
                print(f"Moved anatomy item: {item.name} -> {dst_anatomy}")
            except Exception as e:
                print(f"! Error moving anatomy item {item}: {e}")

# detect experiments like L472_f01, L742_d06, etc.
exp_dirs = [p for p in SRC_ROOT.iterdir() if p.is_dir() and p.name.startswith("L")]
for src_exp in sorted(exp_dirs):
    dst_exp = DST_ROOT / src_exp.name
    make_tree(dst_exp)
    for s_name, d_rel in MAP.items():
        sdir = src_exp / s_name
        if sdir.exists():
            # if this is 00_raw, move "anatomy" first
            if s_name == "00_raw":
                move_anatomy(sdir, dst_exp)
            move_contents(sdir, dst_exp / d_rel)
        else:
            print(f"skip (missing): {sdir}")
    print("OK:", dst_exp)



In [1]:
# Helpers to copy only 03_analysis from each experiment
# \
# Defines:
# find_experiments_with_analysis(src_root): yields experiments that contain a 03_analysis folder.
# copy_analysis_only(...): copies each experiment’s 03_analysis into the destination, with:
# dry_run=True → prints what it would do (no copy)
# overwrite=True → removes an existing destination 03_analysis before copying
# Returns a tuple: (n_experiments_found, n_operations_{planned|done}).

def find_experiments_with_analysis(src_root: Path) -> Iterable[Path]:
    """
    Yield experiment directories under src_root that contain a '03_analysis' subfolder.
    """
    for p in src_root.iterdir():
        if p.is_dir() and (p / "03_analysis").exists():
            yield p


def copy_analysis_only(
        src_root: Path,
        dst_root: Path,
        exp_glob: str = "*",  # pattern to match experiment folder names, e.g. "L4*_Exp_*"
        dry_run: bool = True,  # True = preview; False = actually copy
        overwrite: bool = False,  # True = allow overwriting an existing destination
) -> Tuple[int, int]:
    """
    Copy only the '03_analysis' folder for each experiment folder from src_root to dst_root.

    Layout copied:
      dst_root/<experiment_name>/03_analysis/<...contents...>

    Returns (n_experiments, n_operations_planned).
    """
    src_root = Path(src_root)
    dst_root = Path(dst_root)
    assert src_root.exists(), f"Source root does not exist: {src_root}"

    exps = [p for p in find_experiments_with_analysis(src_root) if p.match(exp_glob)]
    if not exps:
        print("No experiment folders with '03_analysis' found matching pattern.")
        return 0, 0

    ops = 0
    for exp_dir in exps:
        src_analysis = exp_dir / "03_analysis"
        dst_exp = dst_root / exp_dir.name
        dst_analysis = dst_exp / "03_analysis"

        # Plan: create <dst_exp>, then copytree 03_analysis → dst_analysis
        print(f"\n[EXP] {exp_dir.name}")
        print(f"  SRC: {src_analysis}")
        print(f"  DST: {dst_analysis}")

        if dry_run:
            if dst_analysis.exists():
                action = "would OVERWRITE" if overwrite else "would SKIP (exists)"
            else:
                action = "would COPY"
            print(f"  Action (dry-run): {action}")
            ops += 1
            continue

        # Actual copy
        dst_exp.mkdir(parents=True, exist_ok=True)
        if dst_analysis.exists():
            if overwrite:
                print("  Removing existing destination (overwrite=True)...")
                shutil.rmtree(dst_analysis)
            else:
                print("  Destination exists and overwrite=False; skipping.")
                continue

        print("  Copying...")
        shutil.copytree(src_analysis, dst_analysis)  # copies full subtree
        ops += 1

    print(f"\nDone. Experiments found: {len(exps)} | Operations {'planned' if dry_run else 'done'}: {ops}")
    return len(exps), ops


In [2]:
#copy_analysis_only from network to local

# Example: preview or execute the 03_analysis copy
#  Configures src_root and dst_root and calls copy_analysis_only(...).
#
# Important: set dry_run=True for a preview (no files copied).
# When you’re satisfied, run again with dry_run=False to actually copy.
# overwrite=False keeps any existing destination; set True to replace it.
# (In your current snippet, the comment says “Preview” but dry_run=False—flip that to True for a safe preview.)# Set your source and destination roots (Windows UNC / mapped drive paths)

src_root = Path(r"Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro")
dst_root = Path(r"C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p")  # <-- choose where to copy

# 1) Preview (no files copied)
copy_analysis_only(
    src_root=src_root,
    dst_root=dst_root,
    exp_glob="L*",  # or e.g. "L4*_Exp_*" to limit which experiments
    dry_run=False,  # preview first
    overwrite=False,
)
# # 2) Execute copy (set dry_run=False when you're happy with the preview)
# copy_analysis_only(
#     src_root=src_root,
#     dst_root=dst_root,
#     exp_glob="*",  # same pattern you previewed
#     dry_run=False,  # actually copy
#     overwrite=False,  # set True if you want to replace existing destinations
# )



[EXP] L433_f02_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f02_Exp_1_flickering\03_analysis
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f02_Exp_1_flickering\03_analysis
  Destination exists and overwrite=False; skipping.

[EXP] L433_f03_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f03_Exp_1_flickering\03_analysis
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f03_Exp_1_flickering\03_analysis
  Destination exists and overwrite=False; skipping.

[EXP] L433_f04_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f04_Exp_1_flickering\03_analysis
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f04_Exp_1_flickering\03_analysis
  Destination exists and overwrite=False; skipping.

[EXP] L433_f05_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f05_Exp_1_flickering\03_an

(16, 0)

In [3]:
#copy_metadata from network to local

# Function to copy only 01_raw/2p/metadata/**

# Defines copy_metadata_only(...):

# Finds experiments in src_root matching exp_glob.
# If dry_run=True, prints the action it would take.
# If executing, recreates the internal folder structure and copies files into …/01_raw/2p/metadata.
# Respects overwrite: if False, existing files are left as-is.

def copy_metadata_only(
    src_root: Path,
    dst_root: Path,
    exp_glob: str = "*",
    dry_run: bool = True,
    overwrite: bool = False,
):
    """
    Copy only 01_raw/2p/metadata/** for experiments that contain it.
    """
    src_root = Path(src_root); dst_root = Path(dst_root)
    exps = [p for p in src_root.iterdir() if p.is_dir() and p.match(exp_glob)]
    ops = 0
    for exp_dir in exps:
        src_meta = exp_dir / "01_raw" / "2p" / "metadata"
        if not src_meta.exists():
            continue
        dst_meta = dst_root / exp_dir.name / "01_raw" / "2p" / "metadata"
        print(f"\n[EXP] {exp_dir.name}")
        print(f"  SRC: {src_meta}")
        print(f"  DST: {dst_meta}")
        if dry_run:
            if dst_meta.exists():
                action = "would MERGE CONTENTS" if not overwrite else "would OVERWRITE CONTENTS"
            else:
                action = "would COPY CONTENTS"
            print(f"  Action (dry-run): {action}")
            ops += 1
        else:
            dst_meta.mkdir(parents=True, exist_ok=True)
            for root, _, files in os.walk(src_meta):
                rel = Path(root).relative_to(src_meta)
                here = dst_meta / rel
                here.mkdir(parents=True, exist_ok=True)
                for f in files:
                    s = Path(root) / f
                    d = here / f
                    if d.exists() and not overwrite:
                        continue
                    shutil.copy2(s, d)
            ops += 1
    print(f"\nDone. Operations {'planned' if dry_run else 'done'}: {ops}")


In [5]:
# Example: preview or execute the metadata copy

# Calls copy_metadata_only(...) with your src_root/dst_root.
# Tip: use dry_run=True first to confirm paths and actions; then set dry_run=False to perform the copy.
# overwrite=False prevents replacing existing files; set to True if you need to refresh them.

src_root = Path(r"Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro")
dst_root = Path(r"C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p")

# Preview only (prints what it will do; copies nothing)
copy_metadata_only(
    src_root=src_root,
    dst_root=dst_root,
    exp_glob="L*",   # pattern for experiment folder names (use "*" for all)
    dry_run=False,    # preview
    overwrite=False  # don't replace files if they already exist in dst
)



[EXP] L433_f02_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f02_Exp_1_flickering\01_raw\2p\metadata
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f02_Exp_1_flickering\01_raw\2p\metadata

[EXP] L433_f03_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f03_Exp_1_flickering\01_raw\2p\metadata
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f03_Exp_1_flickering\01_raw\2p\metadata

[EXP] L433_f04_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f04_Exp_1_flickering\01_raw\2p\metadata
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f04_Exp_1_flickering\01_raw\2p\metadata

[EXP] L433_f05_Exp_1_flickering
  SRC: Z:\FAC\FBM\CIG\jlarsch\default\D2c\07_Data\Alejandro\L433_f05_Exp_1_flickering\01_raw\2p\metadata
  DST: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f05_Exp_1_flickering\01_